In [1]:
%pylab inline
import scipy.sparse as sp
from scipy.linalg import hadamard as hadamard
from sympy import isprime as isprime
from math import gcd as GCD
from scipy.sparse import diags as sparse
from scipy.sparse.linalg import eigsh as eigval
from scipy.sparse.coo import coo_matrix as sparsemat

Populating the interactive namespace from numpy and matplotlib


In [2]:
    "parameters of the system and operators"

    
    "Number to be decomposed"
    N=21
    
    "control and target registers size"
    L=math.ceil(math.log2(N))
    control_size=2**(2*L)
    target_size=2**L
    
    
    """
    superposition of dimension 2**2L x 2**2L, to be constructed only once.
    Avoided memorization of the H_2L Hadamard operator, since it's not sparse.
    control_superposition= H^2L |0>
    """  
    control_superposition=np.ones(control_size)/control_size
    
    print("qubits required= ",3*L)

qubits required=  15


In [60]:
def find_coprime(N):     
    """
            Find a coprime of N for N>2
            Parameters
            ----------
            N:  Integer
                Number to find the coprime of

    """
    if(N<3):
        raise ValueError("Illegal argument: coprimes exist for N>2")
    Y=randint(2,N)
    used=[]
    while 1:
        a=GCD(Y,N)
        if (a>1):
            #this Y is not coprime
            used.append(Y)
            while(Y in used):
                Y=randint(2,N)
        else: return Y

        
def decimal_to_state(m,nqubit):
    '''
        Return binary representation of m as array of nqubit qubits

            Parameters
            ----------                
            m:   Integer
                number to be representend in binary form
                
            nqubit: Integer
                Total number of qubits used for representation
                    
    '''
    
    arr=binary_repr(m)
    arr=[int(i) for i in arr]
    if(len(arr)>nqubit):
        raise ValueError(str(nqubit)+" are not enough qubits to store the number "+str(m))
    if(len(arr)==nqubit): return arr
    return list(np.zeros(nqubit-len(arr),dtype=int16))+ arr


def to_decimal(array):
    '''
        Return decimal representation of the array storing a number in binary form

        Example: input [1,0,1,0,1] returns 21 
            Parameters
            ----------                
            array: List
                array containing binary representation of a number
                
                    
    '''

    size=len(array)
    return int(np.sum([array[i] *2**(size-i-1)  for i in range(size)]))

In [151]:
def notchosen(chosen,system_size):
    """
            Return array containing the qubit NOT in the partition
            
            Parameters
            ----------                
            chosen:   List
                List of the qubits selected as partition, in the form [1,3,7,..]
                
            system_size: Integer
                Total number of qubits
                    
    """
    
    notchosen=list(set(list(range(system_size)))-set(chosen))
    notchosen.sort()
    return notchosen



def bipartition_state(in_idx,out_idx,chosen,notchosen):
    """
            Return array representing index of the state expressed through in_idx and out_idx
            
            Parameters
            ----------
            in_index:  Integer
                Index of the state restricted on the partition in decimal notation
                
            out_idx:   Integer
                Index of the state restricted on the remaining part in decimal notation    
                       
            chosen:   List
                List of the qubits selected as partition, in the form [1,3,7,..]
                
            notchosen:   List
                List of the qubits not selected as partition, in the form [2,4,5,6,8,..]
                    
    """
        
    part_size=size(chosen)
    out_size=size(notchosen)
    system_size=part_size+out_size
    
    part_state=list(map(lambda x, y:(x,y), decimal_to_state(in_idx,part_size), chosen))
    out_state=list(map(lambda x, y:(x,y), decimal_to_state(out_idx,out_size), notchosen))
   
    total=part_state+out_state
    total.sort(key=lambda x: x[1])
    return [elem[0] for elem in total]
    
def psi(k,Y,N):
    L=int(ceil(log2(N)))
    if(k>2*L):
        raise ValueError(str(k)+"th computational step does not make sense in a "+str(2*L)+" qubits control register")
    
    data=np.ones(2**k,dtype=int8)
    row=[m*2**L+(Y**m%N) for m in range(2**k)]
    col=np.zeros(2**k,dtype=int8)
    psi=sparsemat((data,(row,col)),shape=(2**(k+L),1))
    return (psi/2**k).tocsc()



def create_W(psi,chosen):    
    size=int(ceil(log2(shape(psi)[0])))
    chosen_size=len(chosen)
    not_chosen=notchosen(chosen,size)
    notchosen_size=size-chosen_size
    W=[ [   psi[to_decimal(bipartition_state(i,j,chosen,not_chosen)),0] for j in range(2**notchosen_size)]for i in range(2**chosen_size) ]
    return sparsemat(W,shape=(2**chosen_size,2**notchosen_size))

In [159]:
%%time
kloc,Yloc,Nloc=(10,21,2000)
ps=psi(kloc,Yloc,Nloc)
W=create_W(ps,[0,1])
#k=6, 13 s, k=7 24 s, k=8 49, k=10 circa 3 min e 15 sec

CPU times: user 3min 15s, sys: 756 ms, total: 3min 15s
Wall time: 3min 15s


In [80]:
#coo_matrix((1, m*L+Y**m%N))
row  = np.array([0, 0, 1, 3, 1, 0, 0])
col  = np.array([0, 2, 1, 3, 1, 0, 0])
data = np.array([1, 1, 1, 1, 1, 1, 1])
coo = sparsemat((data, (row, col)), shape=(4, 4))

coo2=sparsemat( ([1],([0],[0])), shape=(4,4)    )

print(coo.toarray())
coo2.toarray()

(coo+coo2).toarray()

[[3 0 1 0]
 [0 2 0 0]
 [0 0 0 0]
 [0 0 0 1]]


array([[4, 0, 1, 0],
       [0, 2, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]], dtype=int64)

In [8]:
print(   sparsemat( ([1],([0],[0])), shape=(4,1)    )        .toarray() )

print(sparsemat(  ([1],([3**2%4],[0])), shape=(2**2,1)).toarray())

[[1]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [0]
 [0]]


In [9]:
print(decimal_to_state(2,4))
print(decimal_to_state(2,3))
np.kron(decimal_to_state(2,4) ,decimal_to_state (2,3)  )

[0.0, 0.0, 1, 0]
[0.0, 1, 0]


array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [74]:
matr=array([array(range(8*i,8*(i+1))) for i in range(0,8)])

In [148]:
log2(2000)

10.965784284662087

In [168]:
test_psi=sparsemat([[0],[0],[1],[1],[0],[0],[1],[1]]).tocsc()
shape(test_psi)
print(create_W(test_psi,[2]).toarray())

[[0 1 0 1]
 [0 1 0 1]]
